# Lecture 9: Models: Simple Linear Regression and Multiple Regression

## Goals
- **Neuroscience / Neuroimaging concepts**
    - How to model and estimate voxel responses to different conditions
    - Simulating a signal
    - Design Matrix
- **Datascience / Coding concepts**
    - Statistical Models
    - Simple Linear Regression
    - Cost functions: Sum of Squared Errors (SSE)
    - Relationship between correlation and regression
    - Multiple Regression

# Setup

Simply run the cells below that contain all the Python modules we'll neeed, plus setup matplotlib for plotting in this jupyter notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats
from scipy.stats import zscore
import nibabel
import cortex
import os
from nistats.hemodynamic_models import glover_hrf as create_hrf

from ipywidgets import interact, interactive, fixed, interact_manual, widgets, FloatSlider

np.random.seed(42)

# Set plotting defaults
%matplotlib inline

## Download data
Run the cell below to download files needed for this lecture

In [ ]:
import urllib.request
import zipfile
zname, _ = urllib.request.urlretrieve("https://berkeley.box.com/shared/static/hcu0khw2spowgd1winlfi4f3si7lk1ua.zip", 
                                      "/home/jovyan/pedagogy_workshop_data.zip")
zfile = zipfile.ZipFile(zname)
zfile.extractall("/home/jovyan")

## Helper Functions
Define a few helper functions that we'll use in this lecture.

In [ ]:
def load_nifti(file_name, mask=None):
    img = nibabel.load(file_name)
    data = img.get_data().T
    if mask is None:
        masked_data = data
    else:
        masked_data = data[:, mask]
    return masked_data

In [ ]:
def plot_line_product(x, y, label_x='X', label_y='Y'):
    plt.plot(x, label=label_x)
    plt.plot(y, label=label_y)
    plt.bar(np.arange(len(x)), x*y, label="Product")
    _ = plt.legend()

In [ ]:
def show_error_time(x, y, slope, intercept):
    """Plots the whole thing as a time series instead of a point cloud"""
    n = x.shape[0]
    plt.plot(y, 'b')
    plt.hlines([0], x.min(), x.max())
    num_x = 1 if x.ndim == 1 else x.shape[1]
    t = np.arange(n)
    new_x = np.zeros((n,))
    for cur_x in range(num_x):
        cur_x = (intercept + slope[cur_x] * x[:,cur_x])
        new_x += cur_x 
    plt.plot(new_x, 'r', lw=1)
    plt.fill_between(t, new_x, y[:,0], color='g', alpha=.5)

In [ ]:
def show_error_scatter(x, y, slope, intercept):
    x = x[:,0]
    y = y[:,0]
    x_left = x.min() - x.ptp() * .2
    x_right = x.max() + x.ptp() * .2
    y_est = slope * x + intercept
    y_left, y_right = slope * x_left + intercept, slope * x_right + intercept
    y_bottom, y_top = y.min() - y.ptp() * .2, y.max() + y.ptp() * .2
    xs = np.vstack([x, x])
    bars = np.vstack([y, y_est])
    plt.hlines([0], x_left, x_right)
    plt.vlines([0], y_bottom, y_top)
    plt.scatter(x, y)
    plt.plot([x_left, x_right], [y_left, y_right], 'r', lw=2)
    plt.plot([0, 0], [0, intercept], 'm', lw=2)
    plt.plot(xs, bars, 'g-.')
    plt.axis([x_left, x_right, y_bottom, y_top])
    return ((y_est - y) ** 2).sum()

In [ ]:
import matplotlib.gridspec as gridspec
def show_error_both_ways(x, y, slope, intercept):
    if x.ndim == 1:
        x = x.reshape(len(x),1)
    if y.ndim == 1:
        y = y.reshape(len(y),1)
    plt.figure(figsize=(20, 8))
    gs = gridspec.GridSpec(1, 5)
    ax = plt.subplot(gs[0, 0:2])
    e = show_error_scatter(x, y, slope, intercept)
    ax = plt.subplot(gs[0, 2:])
    show_error_time(x, y, [slope], intercept)
    print('Error: %.04f' % (e))

# Review


## Correlation

Last week we learned that correlation is a statistic that can be calculated to quantify the similarity of two random variables, and there are multiple ways to think about what exactly it does. We learned three:
* Correlation is the slope of a line that best reduces the squared error between z-scored data in a scatter plot.
* Correlation measures the extent to which two variables change together away from their respective means. 
* Correlation is the average product of the standardized scores of two variables.

Let's create some "fake" data and then review these three ways to think about correlation. First we'll create some constants that we can change later that adjust the size of the fake data and how correlated the two vectors are.

In [ ]:
TARGET_CORRELATION = 0.6
FAKE_N = 50
MEAN_1 = 0
MEAN_2 = 0

Create the fake data with the means and correlation specified above

In [ ]:
fake_data = np.random.multivariate_normal([MEAN_1,MEAN_2], [[1,TARGET_CORRELATION],[TARGET_CORRELATION,1]],FAKE_N)
fake_x = fake_data[:,0]
fake_y = fake_data[:,1]

Let's put that into a function since we'll use it later.

In [ ]:
def create_fake_data(n, mean1, mean2, corr):
    fake_data = np.random.multivariate_normal([mean1,mean2], [[1,corr],[corr,1]],n)
    fake_x = fake_data[:,0]
    fake_y = fake_data[:,1]
    return (fake_x, fake_y)

We learned we can use the `np.corrcoef` function to compute the correlation between two variables. Let's use that to verify that these variables have a correlation coefficient close to what we specified (though it won't be exact due to randomness).

In [ ]:
corr_fake = np.corrcoef(fake_x, fake_y)[0,1]
corr_fake

#### Breakout Session

1\. Rerun the two code cells above that create the random data and calculate their correlation. You should see that the values of the correlation can be pretty different from what we specified.

2\. Now change `FAKE_N` to be `1000` instead of `50`, and rerun the two above code cells several times. How does this change the correlations calculated?

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

### Correlation as the Slope of a Line that Fits Standardized (Z-Scored) Data in a Scatter Plot

When we plot the data from two variables (stored in vectors) on a scatter plot, then the slope of the line that is closest to all the data points, on average, is the correlation. 

First we'll need to z-score both vectors, since correlation is done on standardized data. 

In [ ]:
fake_x_zscore = scipy.stats.zscore(fake_x)
fake_y_zscore = scipy.stats.zscore(fake_y)

Next we'll use a scatter plot to visualize the data of the two variables, which shows the relationship between the data points of two variables. 

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.plot(fake_x_zscore, fake_y_zscore, '.')
plt.xlabel('Fake X')
_ = plt.ylabel('Fake Y')

Now let's add a line to the plot that best fits the line. Since we know that the slope of that line is the correlation, we can plot the line by using the `fake_x` vector as the x data, and multiplying `fake_x` by the correlation for the y data. Let's see.

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.plot(fake_x_zscore, fake_y_zscore, '.')
plt.plot(fake_x_zscore, fake_x_zscore*corr_fake)
plt.xlabel('Fake X')
_ = plt.ylabel('Fake Y')

One important property of plotting the standardized data is that the correlation line always goes through the origin, or (0,0) point, of the plot. This is because the means of both vectors are zero (due to z-scoring), and correlation measures how two signals change together, relative to their respective means.  

#### Breakout Session

1\. Make a figure that is `figsize=(10,5)`, and use `plt.subplot` to make two scatter plots in the same row. The first plot should show the original fake data, and the second the z-scored fake data. Draw appropriate titles above each plot. Do they look similar or different? Why might this be?

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

### Correlation as Two Signals Changing Together, Relative to their Respective Means

If we think of the 2 vectors to be correlated as time series data, then correlation is quantifying how similarily the two signals change at each time point relative to their means (since we z-score the data first, their means are both 0!). Thus if we plot the two z-scored vectors together in a line plot, we can get a visual idea of whether they change together or not. Let's compare the z-scored fake data vectors.

In [ ]:
fig = plt.figure(figsize=(20, 8))
plt.plot(fake_x_zscore, 'g-.', label='zscored X')
plt.plot(fake_y_zscore, 'g', label='zscored Y')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Z-Scored Data')
_ = plt.legend()

They do seem to vary together fairly well, although not perfectly!

#### Breakout Session

1\. Create 2 new sets of fake x and y data, one with a correlation of 0.1 and with a correlation of 0.9.

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

2\. Now z-score both pairs of x and y fake data.

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

3\. Finally create a line plot for each pair of x and y fake data, each on its own row of a subplot.

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

### Correlation as the average product of standardized vectors

We know that when two vectors vary together in time they are correlated. But how do we quantify this? We learned that correlation can be thought of as:

> The average product of two standardized vectors.

In other words, once we have z-scored data, we can simply multiply each pair of data points, and take the average of all of those products. Let's have a look at how this works both visually and algebraically for a few different correlation values.

In [ ]:
fake_x1, fake_y1 = create_fake_data(100, 0, 0, .1)
fake_x2, fake_y2 = create_fake_data(100, 0, 0, .5)
fake_x3, fake_y3 = create_fake_data(100, 0, 0, .9)

And z-score all the data.

In [ ]:
fake_x1_z = zscore(fake_x1)
fake_x2_z = zscore(fake_x2)
fake_x3_z = zscore(fake_x3)
fake_y1_z = zscore(fake_y1)
fake_y2_z = zscore(fake_y2)
fake_y3_z = zscore(fake_y3)

Now let's plot the 3 pairs of fake data, along with their products. We'll use a helper function for this

In [ ]:
fig = plt.figure(figsize=(15,9))
plt.subplot(3,1,1)
plot_line_product(fake_x1_z, fake_y1_z)
plt.title('Correlation .1')
plt.subplot(3,1,2)
plot_line_product(fake_x2_z, fake_y2_z)
plt.title('Correlation .5')
plt.subplot(3,1,3)
plot_line_product(fake_x3_z, fake_y3_z)
_ = plt.title('Correlation .9')

Since we're averaging the products, when the products are all positive we can expect a high positive correlation, which we see in the bottom plot. But when they are both positive and negative they will cancel each other out, resulting in a correlation close to 0, which we see in the top plot.

Now let's calculate the average pairwise product of the two vectors:

In [ ]:
corr_fake1 = np.mean(fake_x1_z*fake_y1_z)
corr_fake2 = np.mean(fake_x2_z*fake_y2_z)
corr_fake3 = np.mean(fake_x3_z*fake_y3_z)

And compare these values to what we get using `np.corrcoef`.

In [ ]:
print(np.allclose(corr_fake1, np.corrcoef(fake_x1_z,fake_y1_z)[0,1]))
print(np.allclose(corr_fake2, np.corrcoef(fake_x2_z,fake_y2_z)[0,1]))
print(np.allclose(corr_fake3, np.corrcoef(fake_x3_z,fake_y3_z)[0,1]))

### Visualizing it all together

Now let's use an interactive plot that plots two data vectors using both a scatter plot and a time series plot to see how both visualizations change together when we add more data, or use more correlated data.

In [ ]:
def plot_cor_time(n):

    # create a sequence for the length of the fake data
    t = range(n)
    
    # subset the fake data to the size specified by the slider
    cur_x = fake_x[:n]
    cur_y = fake_y[:n]

    # z-score (standardize) the fake data
    cur_x_z = scipy.stats.zscore(cur_x)
    cur_y_z = scipy.stats.zscore(cur_y)
    
    # take the mean of the fake data
    mean_fake_x = np.mean(cur_x)
    mean_fake_y = np.mean(cur_y)

    # calculate the correlation of the fake data
    fake_prods = (cur_x-mean_fake_x)*(cur_y-mean_fake_y)
    fake_mean_prods = np.mean(fake_prods)
    fake_prod_std = np.std(cur_x) * np.std(cur_y)
    fake_cor = fake_mean_prods / fake_prod_std

    # print out the values of the correlation
#    print('Average of products: %.02f' % (fake_mean_prods))
#    print('Product of std-dev: %.02f' % (fake_prod_std))
    print('Correlation: %.02f' % (fake_cor))

    # plot the line plots of the two variables
    plt.figure(figsize=(15,6))
    grid = plt.GridSpec(2,5)
    plt.subplot(grid[0,2:])
    plt.plot(t, cur_x, 'g', label='x')
    plt.plot(t, cur_x, 'go')
    plt.plot(t, cur_y, 'b', label='y')
    plt.plot(t, cur_y, 'bo')
    plt.plot(t, np.ones(n)*mean_fake_x, 'g--')
    plt.plot(t, np.ones(n)*mean_fake_y, 'b--')
    plt.legend()

    # plot the product of the difference between each data point and their respective means
    plt.subplot(grid[1,2:])
    plt.plot(t, np.ones(n)*fake_mean_prods, '--')
    plt.bar(t,fake_prods)

    # make a scatter plot of the standardized data, and draw the correlation line
    plt.subplot(grid[0:2,0:2])
    plt.scatter(cur_x_z, cur_y_z)
    fit = np.polyfit(cur_x_z, cur_y_z, deg=1)
    plt.plot(cur_x_z, fit[0] * cur_x_z + fit[1], color='red')

In [ ]:
interactive_plot = interactive(plot_cor_time, n=widgets.IntSlider(value=2, min=2, max=len(fake_x)))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

## Correlation quantifies linear dependency

Earlier in the lecture we were using correlation coefficient to assess how much two sets of numbers relate to each other. There are some assumptions behind this, which we will take a look at here.

The general intuition, when we say **"these quantities are correlated"**, it means: There is some form of **dependency** between the two quantities. Meaning: knowledge of the one quantity might gain us more knowledge about the other quantity. This is a **loose definition**, but there exist mathematical ideas to quantify this.

### When we say *correlation*, we usually refer to *linear correlation* (also called Pearson correlation)
Or to frame it as a question: **How much does the point cloud resemble a line?**

We saw above that when 2 vectors are linearly related, without noise, they formed a perfect line, meaning a perfect positive correlation of 1. Let's see that again.

In [ ]:
x = np.random.randn(100)
y = 2 * x + 4
plt.scatter(x, y)
_ = plt.axis([-3, 3, -11, 11])

In [ ]:
np.corrcoef(x, y)

### Linear Correlation measured on a non-linear relation

There can be many different **non-linear** ways that two vectors can relate to each other, or be dependent on each other. One simple way is when one vector is the square of the other, resulting in a parabola when plotted on a scatter plot. Let's create a `height_squared` vector that shows just that. 

In [ ]:
x = np.linspace(-1., 1., 51)
y = x ** 2
plt.plot(x, y, 'o')

We see that these two vectors create a parabola when plotted on a scatter plot! While there is definitely a precise relationship between the two arrays, correlation cannot detect that relationship because it only looks for linear relationships. Let's look at what the correlation between these vectors is.

In [ ]:
np.corrcoef(x, y)

Basically exactly zero! Let's plot the correlation line along with the data

In [ ]:
x_z = (x - x.mean()) / x.std()
y_z = (y - y.mean()) / y.std()
plt.plot(x_z, y_z, 'o')
c = np.mean(x_z * y_z)
plt.plot(x_z, c * y_z)

The correlation coefficient of this *perfect functional relation* is **zero**. Always keep this in mind when thinking about and evaluating correlation scores.

## Creating Correlation Flatmaps

In this section we will correlate all the voxels of a scan with the response vectors for the experiment. We will compute all of them together using array operations, and eventually even compute all of them together for stimulus types. Then we will display the results on flat maps.

### Load the visual category fMRI data

We'll start by loading the visual category localizer data that we've been using.

In [ ]:
data01 = load_nifti("/data/cogneuro/fMRI/categories/s01_categories_01.nii.gz")

In [ ]:
faces_voxel_1 = scipy.stats.zscore(data01[:, 6, 57, 37])
data_faces = faces_voxel_1

In [ ]:
places_voxel_1 = scipy.stats.zscore(data01[:, 10, 62, 40])
data_places = places_voxel_1

Now load the stimulus category labels and create the stimulus vectors for all the categories

In [ ]:
localizer_labels = np.load("/home/jovyan/catloc_experimental_conditions.npy")
faces_stimuli_vec = localizer_labels == 'faces'
bodies_stimuli_vec = localizer_labels == 'body'
places_stimuli_vec = localizer_labels == 'places'
objects_stimuli_vec = localizer_labels == 'object'
scrambled_stimuli_vec = localizer_labels == 'scrambled'

In [ ]:
faces_stimuli_vec = faces_stimuli_vec[:data_places.shape[0]]
bodies_stimuli_vec = bodies_stimuli_vec[:data_places.shape[0]]
places_stimuli_vec = places_stimuli_vec[:data_places.shape[0]]
objects_stimuli_vec = objects_stimuli_vec[:data_places.shape[0]]
scrambled_stimuli_vec = scrambled_stimuli_vec[:data_places.shape[0]]

We'll create an HRF, and convolve it with each of the stimulus vectors to create the response vectors.

In [ ]:
# create the HRF function
hrf = create_hrf(tr=2.0, oversampling=1, time_length=32)

# create the response vectors
faces_response_vec = np.convolve(faces_stimuli_vec, hrf)[:len(faces_stimuli_vec)]
bodies_response_vec = np.convolve(bodies_stimuli_vec, hrf)[:len(bodies_stimuli_vec)]
places_response_vec = np.convolve(places_stimuli_vec, hrf)[:len(places_stimuli_vec)]
objects_response_vec = np.convolve(objects_stimuli_vec, hrf)[:len(objects_stimuli_vec)]
scrambled_response_vec = np.convolve(scrambled_stimuli_vec, hrf)[:len(scrambled_stimuli_vec)]

And pull out the number of categories in the data, and create a vector that represents time for the data as well.

In [ ]:
n_cat = len(faces_stimuli_vec)
t_cat = np.arange(0,(n_cat*2),2)

### Preparing the data

Above we already loaded the BOLD data into the names `data1`. Let's check their shapes:

In [ ]:
data01.shape

Let's mask them to obtain the cortical voxels.

In [ ]:
mask = cortex.db.get_mask('s01', 'catloc', 'cortical')

In [ ]:
data1_masked = data01[:, mask]

Now let's perform z-scoring for the first array for all the voxels separately:

First, we compute the mean of each voxel time series:

In [ ]:
data1_masked.shape

In [ ]:
data1_mean = data1_masked.mean(axis=0)

In [ ]:
data1_mean.shape

We see that taking the mean along axis 0 gives us a mean value for each voxel.

We can also take the standard deviation along this axis:

In [ ]:
data1_stdev = data1_masked.std(axis=0)
data1_stdev.shape

Now we can do the z-scoring by subtracting the mean and dividing by the standard deviation for each column of the masked data array:

In [ ]:
data1_zscored = (data1_masked - data1_mean) / data1_stdev
data_zscored = data1_zscored

### Computing correlation of one response vector to all the voxels

In order to compute the correlation of a response vector to all the voxels, we will first spell out the multiplication of that one response vector to all voxels.

In [ ]:
faces_response_zscored = zscore(faces_response_vec)
faces_response_times_voxels = faces_response_zscored * data_zscored

#### Breakout Session
1. Do z-scoring for the other two arrays
2. Use `np.concatenate` or `np.vstack` to stack all three matrices vertically into an array named `data_zscored`.

Huh, that didn't work!

To make this work, we need to turn `faces_response_zscored` into a column vector of shape `(360, 1)`.

In [ ]:
faces_response_zscored_column = faces_response_zscored.reshape(-1, 1)

In [ ]:
faces_response_times_voxels = faces_response_zscored_column * data_zscored

That worked. Now we can take the mean along the time axis to compute the correlations with each voxel.

In [ ]:
corr_faces_data = faces_response_times_voxels.mean(axis=0)

In [ ]:
corr_faces_data.max(), corr_faces_data.min()

In [ ]:
vol_faces = cortex.Volume(corr_faces_data, 's01', 'catloc', vmin=-.5, vmax=.5)
_ = cortex.quickflat.make_figure(vol_faces)

In [ ]:
vol_faces = cortex.Volume(corr_faces_data, 's01', 'catloc', vmin=-.5, vmax=.5)
cortex.webshow(vol_faces)

We find that the `faces_response_zscored` correlates most strongly with voxels that we have already identified as belonging to areas processing faces!

#### Breakout Session
1. Compute the same correlation for `places` in the PPA voxel, and make a flatmap showing the correlation values. 

In [ ]:
### STUDENT ANSWER

### END STUDENT ANSWER

# Linear Regression

We've seen that correlation is a statistical technique that quantifies the linear relationship between two vectors of data by calculating the correlation coefficient, $r$. One way to think about correlation is that it finds a line that best fits the data when the standardized (z-scored) data is plotted in a scatter plot. **Linear regression** is also a statistical technique that finds the line that best fits data when plotted in a scatter plot, but it does not require that the data is standardized. 

## Independent vs. Dependent Variables
Another way to describe linear regression is that it finds the linear relationship between a single **dependent variable** (y) and one or more **independent variables** denoted (x). Thus **linear regression** is more powerful than correlation because it can quantify the relationship of many different **independent variables** with the **dependent variable**, whereas correlation only finds the relationship between two **variables**, neither of which is independent nor dependent. 

* **Dependent Variable**: This variable represents the data you've measured, and are trying to understand in terms of the independet variables. In our case, the BOLD data time series is the dependent variable.
* **Independent Variable**: These are the one or more variables that we're using to explain the dependent variable. These are generally not measured, rather their true values are known. In our case, the response vectors of all the visual categories or motor tasks are the independent variables. 

## The linear equation: `y = a*x + b`

We saw last week that we can create a data vector $y$ that is perfectly correlated with a data vector $x$ by using the linear equation: $$y = a*x+b$$

Then we can add noise to $y$ to decrease that correlation. The correlation coefficient, $r$, was neither $a$ nor $b$ however. 

Linear regression gives us a tool to find $a$ and $b$ if the correlation between $x$ and $y$ is perfect, or to estimate the values of $a$ and $b$ that make the average of all the linearly transformed values of $x$ as close as possible to the values of $y$.

So let's use some simple fake data to get a feeling for what $a$ and $b$ are using plots. We'll create 4 pairs of fake data with different values of $a$ and $b$.

In [ ]:
def create_fake_linear(n, a, b):
    x = np.random.randn(n)
    y = a * x + b
    return (x,y)

In [ ]:
fake_linear_x1, fake_linear_y1 = create_fake_linear(100, 1, 0)
fake_linear_x2, fake_linear_y2 = create_fake_linear(100, 1, 4)
fake_linear_x3, fake_linear_y3 = create_fake_linear(100, 2, 0)
fake_linear_x4, fake_linear_y4 = create_fake_linear(100, .3, -3)

Let's plot the four pairs of fake data onto a plot to see how changing $a$ and $b$ changes the line.

**NOTE**: Because we want to get a sense for the real slope of the line we must make sure the x and y axes are on the same scale. To do this we can use `plt.axis('square')`. If you don't do this the scales of the two axes will be automatically calculated by default and the angle of the line won't be the actual slope.

In [ ]:
fig = plt.figure(figsize=(8,7))
plt.plot(fake_linear_x1, fake_linear_y1, '.', label='a=1 b=0')
plt.plot(fake_linear_x2, fake_linear_y2, '.', label='a=1 b=4')
plt.plot(fake_linear_x3, fake_linear_y3, '.', label='a=2 b=0')
plt.plot(fake_linear_x4, fake_linear_y4, '.', label='a=0.3 b=-3')
plt.grid()
plt.axis('square')
_ = plt.legend()

#### Breakout Session

1\. Think back to algebra class and determine what the slope and intercept are for the 4 fake data sets plotted in the cell above.

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

### Estimating the Slope: `a`

The slope of a line is it's angle, which represents the change in y for every change of 1 in x. It's often termed "Rise over Run" because it can be calculated be choosing any two points on the line and dividing the difference in y-values of those two points (the Rise) by the difference in the x-values (the Run).

In regression, we use the value of one or more variables (x) to predict the value of another (y). When the variables x and y are measured in standard units, the regression line for predicting y based on x has slope r and passes through the origin. Thus the equation of the regression line can be written as:

$$\hat{y} = r*x$$

where $r$ is the correlation coefficient, $\hat{y}$ is the **predicted value of y**, and when both $x$ and $y$ variables are measured in standard units (z-scored).
 
Generally we don't want to have to z-score the data in order to find a relationship between $x$ and $y$, and so we can write the linear regression equation in terms of the original units of the data. We do this by inverting the z-score operation, namely multiplying both variables by their respective standard deviations, and adding back in their respective means:

$$\frac{\hat{y} − \bar{y}}{SD_y} = r * \frac{x - \bar{x}}{SD_x}$$

The figures below give a visual representation of how the we arrive at the above equation.

<img src="figures/regline_z.png" align="left" style="height: 200px;"/>
<img src="figures/regline_orig.png" align="right" style="height: 200px;"/>

By rearranging the terms in the equation above, we can retrieve the equation for estimating the slope of the regresion line:
$$slope = r*\frac{SD_y}{SD_x}$$
 
where $r$ is the correlation coefficient, SD_y is the standard deviation of y, and SD_x is the standard deviation of x. 

Let's put this equation into a function and use it to calculate the slope of the regression line for the fake data sets we just created.

In [ ]:
def calc_slope(x, y):
    r = np.corrcoef(x, y)[0,1]
    return r*np.std(y)/np.std(x)

In [ ]:
slope_fake1 = calc_slope(fake_linear_x1, fake_linear_y1)
slope_fake2 = calc_slope(fake_linear_x2, fake_linear_y2)
slope_fake3 = calc_slope(fake_linear_x3, fake_linear_y3)
slope_fake4 = calc_slope(fake_linear_x4, fake_linear_y4)
print(slope_fake1,slope_fake2,slope_fake3,slope_fake4)

Ok, we were able to find the slopes perfectly since there was no noise! Now let's update the function to create fake data by adding noise to it, and see how we do.

In [ ]:
def create_fake_linear(n, a, b, noise_sd=None):
    x = np.random.randn(n)
    y = a * x + b
    if noise_sd is not None:
        y += np.random.randn(n)*noise_sd
    return (x,y)

And now we'll create some fake linear data that has noise.

In [ ]:
fake_linear_x5,fake_linear_y5 = create_fake_linear(100, 1, 0, .3)
fake_linear_x6,fake_linear_y6 = create_fake_linear(100, 2, 0, .3)
fake_linear_x7,fake_linear_y7 = create_fake_linear(100, .3, 0, .3)

Now we'll use the slope function to estimate the slope that best fits this data, but it won't be perfect because we've added noise.

In [ ]:
slope_fake5 = calc_slope(fake_linear_x5, fake_linear_y5)
slope_fake6 = calc_slope(fake_linear_x6, fake_linear_y6)
slope_fake7 = calc_slope(fake_linear_x7, fake_linear_y7)
print('True Value: %.03f, Estimated Value: %.03f' % (1, slope_fake5))
print('True Value: %.03f, Estimated Value: %.03f' % (2, slope_fake6))
print('True Value: %.03f, Estimated Value: %.03f' % (.3, slope_fake7))

That's still pretty good, the estimated values for the slope are very close to the original values! 

Let's plot the data along with the regression line to visualize what's happening. Note that fake data sets 5, 6 & 7 were created with an intercept of 0, so we don't need to estimate an intercept for these plots to look good. Normally you would, and we'll learn how to do that next.

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot(fake_linear_x5, fake_linear_y5, '.')
plt.plot(fake_linear_x5, fake_linear_x5 * slope_fake5)
plt.axis('square')
plt.title('Fake Data 5')
plt.subplot(1,3,2)
plt.plot(fake_linear_x6, fake_linear_y6, '.')
plt.plot(fake_linear_x6, fake_linear_x6 * slope_fake6)
plt.axis('square')
plt.title('Fake Data 6')
plt.subplot(1,3,3)
plt.plot(fake_linear_x7, fake_linear_y7, '.')
plt.plot(fake_linear_x7, fake_linear_x7 * slope_fake7)
plt.axis('square')
_ = plt.title('Fake Data 7')

#### Breakout Session

1\. Create a new pair of fake $x$ and $y$ that is 100 long called `fake_breakout_x` and `fake_breakout_y`, with `a=3.2`, `b=4.2` and `noise_sd=10`. 

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

2\. Now calculate the slope for a linear regression between these two new variables. How close is it to the original `a`? Why is it close or far from the original value?

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

### Estimating the Intercept: `b`

The intercept is the point on the y axis where the regression line intersects it. We can estimate it using the following equation derived in the same manner as that for the slope:

$$intercept = \bar{y} − slope*\bar{x}$$ 

where $\hat{y}$ is the mean of $y$, and $\hat{x}$ is the mean of $x$.

Let's put that equation into a helper function and use it on our fake data.

In [ ]:
def calc_intercept(x, y):
    return np.mean(y) - calc_slope(x, y) * np.mean(x)

In [ ]:
intercept_fake1 = calc_intercept(fake_linear_x1, fake_linear_y1)
intercept_fake2 = calc_intercept(fake_linear_x2, fake_linear_y2)
intercept_fake3 = calc_intercept(fake_linear_x3, fake_linear_y3)
intercept_fake4 = calc_intercept(fake_linear_x4, fake_linear_y4)

print('Real Intercept: %.03f, Estimated Intecept: %.03f' % (0,intercept_fake1))
print('Real Intercept: %.03f, Estimated Intecept: %.03f' % (4,intercept_fake2))
print('Real Intercept: %.03f, Estimated Intecept: %.03f' % (0,intercept_fake3))
print('Real Intercept: %.03f, Estimated Intecept: %.03f' % (-3,intercept_fake4))

Again we did perfect because there was no noise in this data. Let's add some noise and see how we do.

In [ ]:
fake_linear_x8, fake_linear_y8 = create_fake_linear(100, 1.5, 5, .5)
fake_linear_x9, fake_linear_y9 = create_fake_linear(100, 5, -.4, .5)
fake_linear_x10, fake_linear_y10 = create_fake_linear(100, -2, 2, .5)

We'll estimate both the slope and intercept now so we can plot the data with the regression line.

In [ ]:
intercept_fake8 = calc_intercept(fake_linear_x8, fake_linear_y8)
intercept_fake9 = calc_intercept(fake_linear_x9, fake_linear_y9)
intercept_fake10 = calc_intercept(fake_linear_x10, fake_linear_y10)

slope_fake8 = calc_slope(fake_linear_x8, fake_linear_y8)
slope_fake9 = calc_slope(fake_linear_x9, fake_linear_y9)
slope_fake10 = calc_slope(fake_linear_x10, fake_linear_y10)

print('Real Intercept: %.03f, Estimated Intecept: %.03f' % (5,intercept_fake8))
print('Real Intercept: %.03f, Estimated Intecept: %.03f' % (-.4,intercept_fake9))
print('Real Intercept: %.03f, Estimated Intecept: %.03f' % (2,intercept_fake10))


Again we did a good job! Let's see what it looks like.

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.subplot(1,3,1)
plt.plot(fake_linear_x8, fake_linear_y8, '.')
plt.plot(fake_linear_x8, fake_linear_x8 * slope_fake8 + intercept_fake8)
plt.axis('square')
plt.title('Fake Data 8')
plt.subplot(1,3,2)
plt.plot(fake_linear_x9, fake_linear_y9, '.')
plt.plot(fake_linear_x9, fake_linear_x9 * slope_fake9 + intercept_fake9)
plt.axis('square')
plt.title('Fake Data 9')
plt.subplot(1,3,3)
plt.plot(fake_linear_x10, fake_linear_y10, '.')
plt.plot(fake_linear_x10, fake_linear_x10 * slope_fake10 + intercept_fake10)
plt.axis('square')
_ = plt.title('Fake Data 10')

#### Breakout Session

1\. Estimate the intercept of a linear regression of `fake_breakout_x` onto `fake_breakout_y`. Is it close to the original `b`? Why or why not?

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

## Relationship between Correlation and Simple Linear Regression

As was suggested earlier, there is a very intersting relationship between correlation and the slope in a **Simple Linear Regression** (which is a regression with only 1 independent variable). 

If we standardize (z-score) both our x (independent) and y (dependent) variables, then the slope we find using linear regression is exactly the same as the correlation, and the intercept is 0. Let's have a look, first we'll z-score our fake data.

In [ ]:
fake_linear_x10_z = scipy.stats.zscore(fake_linear_x10)
fake_linear_y10_z = scipy.stats.zscore(fake_linear_y10)

Now let's calculate the correlation, slope and intercept for the z-scored data.

In [ ]:
fake_intercept10_z = calc_intercept(fake_linear_x10_z, fake_linear_y10_z)
fake_slope10_z = calc_slope(fake_linear_x10_z, fake_linear_y10_z)
fake_correlation10_z = np.corrcoef(fake_linear_x10_z, fake_linear_y10_z)[0,1]

# print out the results
print('Linear Regression Intercept: %.06f' % (fake_intercept10_z))
print('Linear Regression Slope: %.06f' % (fake_slope10_z))
print('Correlation: %.06f' % (fake_correlation10_z))

#### Breakout Session

1\. Now z-score `fake_breakout_x` and `fake_breakout_y` and recalculate the slope on these z-scored vectors. Confirm this gives you a correlation by calculating the correlation between the original (non-zscored) vectors.

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER

## Finding the line which minimizes error

Now that we have an idea of what the slope and intercept are, let's get an intuitive sense for what values will give a good "fit" to the data using some interactive plots. This plot has two sliders, one for the slope and one for the intercept. Moving the sliders will change the slope and intercept of the red line in the plot. The data points are draw in blue, and there are lines between each data point and the regression line in green. You want the sum of all of these lines to be as small as possible. That sum is printed as a number beneath the plot. 

Try to adjust the slope and intercept to find the minimum value of that sum at the bottom, which is exactly what linear regression does!

In [ ]:
def show_error_fake(slope, intercept):
    plt.figure(figsize=(7,7))
    return show_error_scatter(fake_linear_x10.reshape(-1,1), fake_linear_y10.reshape(-1,1), slope, intercept)

_ = interact(show_error_fake, 
             slope=FloatSlider(value=0, min=-5, max=5), 
             intercept=FloatSlider(value=0, min=-5, max=5))

### Visualizing Linear Regression in Two Ways

We learned that we can plot two vectors as time series on a line plot to get an idea of how correlated they are. We can do a very similar thing to try and guess the values for the slope and intercept of a linear regression between two vectors. This interactive plot is the same as the last one, with the addition of a time series line plot. Have a look!

In [ ]:
fake_linear_x10.reshape(100,1).shape

In [ ]:
def show_error_fake10_both(slope, intercept):
    return show_error_both_ways(fake_linear_x10, fake_linear_y10, slope, intercept)

interactive_plot = interactive(show_error_fake10_both, 
                               slope=widgets.FloatSlider(value=0, min=-5, max=5), 
                               intercept=widgets.FloatSlider(value=0, min=-5, max=5))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

In [ ]:
plt.plot(fake_linear_x10)

#### Breakout Session

1\. What operations are applied to the response vector when the 'slope' and 'intersept' sliders are changed? Why does changing the values cause the error value to increase or decrease?

In [ ]:
### STUDENT ANSWER
### END STUDENT ANSWER